# что делать дальше

2. смотрим старые решения

1. открываем старые скрипты где может удобно все сделанно
2. настраиваем гиперпараметры и сравниваем модели

In [1]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
import sklearn.linear_model as LR
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

import lightgbm as lgb
import xgboost as xgb

import joblib
import optuna

In [2]:
df = pd.read_csv('../data/processed/prepared_features.csv')
#df = df[df['Quality'] != 1]

In [31]:

# Split the data
X, y  = df.drop(['Yield', 'Field_ID', 'Quality'], 1), df['Yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
X_train = X_train.astype(float)
# Fit the model
model = RandomForestRegressor(random_state = 42)
model.fit(X_train, y_train)

# Score with RMSE
print('RFR, Score:', mean_squared_error(y_test, model.predict(X_test), squared=False))

model = ExtraTreesRegressor(random_state = 42)
model.fit(X_train, y_train)

# Score with RMSE
print('ETR, Score:', mean_squared_error(y_test, model.predict(X_test), squared=False))

model = lgb.LGBMRegressor(random_state = 42)
model.fit(X_train, y_train)

# Score with RMSE
print('Lightgbm, Score:', mean_squared_error(y_test, model.predict(X_test), squared=False))

model = xgb.XGBRegressor(random_state = 42)
model.fit(X_train, y_train)

# Score with RMSE
print('XGBRegressor, Score:', mean_squared_error(y_test, model.predict(X_test), squared=False))

model = LR.LinearRegression()
model.fit(X_train, y_train)

# Score with RMSE
print('LR, Score:', mean_squared_error(y_test, model.predict(X_test), squared=False))

RFR, Score: 1.5265292629983735
ETR, Score: 1.5738586947938809
Lightgbm, Score: 1.5459168763347544
XGBRegressor, Score: 1.6307029478404718
LR, Score: 1.493667190070125


# cross val check

In [47]:
X, y  = df[df['Quality']!=1].drop(['Yield', 'Field_ID', 'Quality', 'year'], 1), df['Yield']

In [46]:
X, y  = df.drop(['Yield', 'Field_ID', 'Quality', 'year'], 1), df['Yield']

In [48]:
model = lgb.LGBMRegressor(random_state = 42)

In [34]:
usefull_period_col = [col for col in X.columns if col[0] in ['3', '4', '5', '6', '7', '9']]

In [19]:
usefull_period_col

['3_S2_B8',
 '3_S2_B4',
 '3_S2_B2',
 '3_S2_B3',
 '3_S2_B9',
 '3_S2_QA60',
 '3_7_CLIM_pet',
 '3_CLIM_pr',
 '4_S2_B8',
 '4_S2_B4',
 '4_S2_B2',
 '4_S2_B3',
 '4_S2_B9',
 '4_S2_QA60',
 '4_7_CLIM_pet',
 '4_CLIM_pr',
 '5_S2_B8',
 '5_S2_B4',
 '5_S2_B2',
 '5_S2_B3',
 '5_S2_B9',
 '5_S2_QA60',
 '5_7_CLIM_pet',
 '5_CLIM_pr',
 '5_ndvi',
 '5_evi',
 '5_grvi',
 '5_gndvi',
 '5_wrdvi',
 '4_ndvi',
 '4_evi',
 '4_grvi',
 '4_gndvi',
 '4_wrdvi',
 '3_ndvi',
 '3_evi',
 '3_grvi',
 '3_gndvi',
 '3_wrdvi']

In [49]:
xval_err = 0
kf = KFold(n_splits=7)
X_temp = X[usefull_period_col]
for train, test in kf.split(X_temp):
    model.fit(X_temp.iloc[train],y.iloc[train])
    pred = model.predict(X_temp.iloc[test])
    err = mean_squared_error(y.iloc[test], pred)
    xval_err+=err
rmse_cv = xval_err/10
rmse_cv

2.453459012979943

on all cols 1.97

RFR

In [50]:
model = RandomForestRegressor(random_state = 42, n_jobs = -1)
kf = KFold(n_splits=7)

In [51]:
usefull_period_col = [col for col in X.columns if col[0] in ['3', '4', '5', '6']]

In [67]:
X_f_month = X[usefull_period_col].copy()

In [71]:
s2_cols = [col for col in X_f_month.columns if col.split('_')[-1] in ['B9', 'B2']]

In [72]:
s2_cols

['3_S2_B2',
 '3_S2_B9',
 '4_S2_B2',
 '4_S2_B9',
 '5_S2_B2',
 '5_S2_B9',
 '6_S2_B2',
 '6_S2_B9']

In [73]:
xval_err = 0
kf = KFold(n_splits=5)
X_temp = X.drop(s2_cols, 1)
for train, test in kf.split(X_temp):
    model.fit(X_temp.iloc[train],y.iloc[train])
    pred = model.predict(X_temp.iloc[test])
    err = mean_squared_error(y.iloc[test], pred)
    xval_err+=err
rmse_cv = xval_err/10
rmse_cv

1.6550001814288204

on all cols 1.9

on monthes - ['3', '4', '5', '6'] - 1.31 + 

if drop quality - 1 then 1.61 +

if drop quality - 3 then 1.68

drop S2 - 1.64

drop - ['B8', 'B9', 'B2'] - 1.66

drop - ['B9', 'B2'] - 1.66

# hyperparam tuning

In [68]:
def validate_cv(params):
    kf = KFold(n_splits=7)
    model = RandomForestRegressor(**params, 
                                  random_state = 42, 
                                  n_jobs = -1)
    
    xval_err = 0
    for train, test in kf.split(X):
        model.fit(X.iloc[train],y.iloc[train])
        pred = model.predict(X.iloc[test])
        err = mean_squared_error(y.iloc[test], pred)
        xval_err+=err
    rmse_cv = xval_err/10
    return rmse_cv

def objective(trial):
    list_trees = [25, 50, 75, 100, 125, 150, 175, 200, 225, 250]

    params = {'n_estimators': trial.suggest_categorical('n_estimators', list_trees),
              'max_features': trial.suggest_int('max_features', 3, 30),
              'min_samples_split': trial.suggest_int('min_samples_split', 2, 14),
              'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
              'max_samples': trial.suggest_uniform('max_samples', 0.6, 0.99)}
    
    
    return validate_cv(params)

In [61]:
def validate_cv(param):
    kf = KFold(n_splits=7)
    xval_err = 0
    for train, test in kf.split(X):
        dtrain = lgb.Dataset(X.iloc[train],label = y.iloc[train])
        gbm = lgb.train(param, dtrain)
        
        pred = gbm.predict(X.iloc[test])
        err = mean_squared_error(y.iloc[test], pred)
        xval_err+=err
    rmse_cv = xval_err/10
    return rmse_cv

def objective(trial):

    param = {
        'objective': 'regression',
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "learning_rate": trial.suggest_uniform("learning_rate", 0.001, 0.15),
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'rmse', valid_name='valid_1')   
    #gbm = lgb.train(param, dtrain)
    return validate_cv(param)

In [62]:
X, y  = df.drop(['Yield', 'Field_ID', 'Quality', 'year'], 1), df['Yield']

In [63]:
lgbm_study = optuna.create_study(direction="minimize")
# study = joblib.load('../optuna_checkpoints/study.pkl')
lgbm_study.optimize(objective, n_trials=300)
joblib.dump(lgbm_study, '../optuna_checkpoints/lgbm_study_with_bands_and_low_q.pkl')

[I 2020-11-01 12:26:31,284] A new study created in memory with name: no-name-2458299b-d555-4b66-bd35-fc044abca270
[I 2020-11-01 12:26:33,306] Trial 0 finished with value: 1.7812338515378348 and parameters: {'learning_rate': 0.002587329429388989, 'lambda_l1': 1.3190669043573704, 'lambda_l2': 0.0004576381610477084, 'num_leaves': 13, 'feature_fraction': 0.8989916634954236, 'bagging_fraction': 0.9200396549182275, 'bagging_freq': 3, 'min_child_samples': 62}. Best is trial 0 with value: 1.7812338515378348.
[I 2020-11-01 12:26:34,563] Trial 1 finished with value: 1.604029915230134 and parameters: {'learning_rate': 0.07184389971134258, 'lambda_l1': 0.01133199356160213, 'lambda_l2': 0.16639673662631, 'num_leaves': 181, 'feature_fraction': 0.45404141460512015, 'bagging_fraction': 0.6751397667832113, 'bagging_freq': 6, 'min_child_samples': 79}. Best is trial 1 with value: 1.604029915230134.
[I 2020-11-01 12:26:36,020] Trial 2 finished with value: 1.635378994290674 and parameters: {'learning_rate'

[I 2020-11-01 12:27:27,321] Trial 20 finished with value: 1.5740523085811746 and parameters: {'learning_rate': 0.02684657966565308, 'lambda_l1': 6.588563671204393e-05, 'lambda_l2': 0.0021743679133357538, 'num_leaves': 106, 'feature_fraction': 0.5109118234158441, 'bagging_fraction': 0.5451608950475896, 'bagging_freq': 7, 'min_child_samples': 39}. Best is trial 16 with value: 1.5601602704808706.
[I 2020-11-01 12:27:28,760] Trial 21 finished with value: 1.8411975789749526 and parameters: {'learning_rate': 0.0013371065679603396, 'lambda_l1': 0.002205629639697269, 'lambda_l2': 7.383242460484999e-07, 'num_leaves': 70, 'feature_fraction': 0.5119372065001894, 'bagging_fraction': 0.5052020948483245, 'bagging_freq': 2, 'min_child_samples': 49}. Best is trial 16 with value: 1.5601602704808706.
[I 2020-11-01 12:27:29,856] Trial 22 finished with value: 1.566774043289005 and parameters: {'learning_rate': 0.026613900635465912, 'lambda_l1': 5.653495694624423e-05, 'lambda_l2': 1.333651969915562e-07, 'n

[I 2020-11-01 12:27:55,966] Trial 40 finished with value: 1.6679467684492502 and parameters: {'learning_rate': 0.1008611499042355, 'lambda_l1': 2.1762736283704023e-06, 'lambda_l2': 0.005911813496788606, 'num_leaves': 116, 'feature_fraction': 0.7228010806541497, 'bagging_fraction': 0.4782205394061416, 'bagging_freq': 4, 'min_child_samples': 65}. Best is trial 16 with value: 1.5601602704808706.
[I 2020-11-01 12:27:57,117] Trial 41 finished with value: 1.5628741420772767 and parameters: {'learning_rate': 0.028062136608711284, 'lambda_l1': 1.5539239105487886e-05, 'lambda_l2': 1.8079689356104143e-05, 'num_leaves': 84, 'feature_fraction': 0.6289774875945227, 'bagging_fraction': 0.5285684540240201, 'bagging_freq': 2, 'min_child_samples': 95}. Best is trial 16 with value: 1.5601602704808706.
[I 2020-11-01 12:27:58,793] Trial 42 finished with value: 1.593419049288666 and parameters: {'learning_rate': 0.04330230765465787, 'lambda_l1': 7.515902664197494e-06, 'lambda_l2': 1.7906389009861356e-05, '

[I 2020-11-01 12:28:26,125] Trial 60 finished with value: 1.5576307654501302 and parameters: {'learning_rate': 0.02318585548974285, 'lambda_l1': 0.0014606737605343267, 'lambda_l2': 1.2018918698089092, 'num_leaves': 108, 'feature_fraction': 0.7099100895128319, 'bagging_fraction': 0.4926159900613805, 'bagging_freq': 2, 'min_child_samples': 77}. Best is trial 60 with value: 1.5576307654501302.
[I 2020-11-01 12:28:27,424] Trial 61 finished with value: 1.5605097811188227 and parameters: {'learning_rate': 0.021406009721537205, 'lambda_l1': 0.0069476973498815596, 'lambda_l2': 0.5264800794032386, 'num_leaves': 110, 'feature_fraction': 0.8059022075786096, 'bagging_fraction': 0.4605670358015993, 'bagging_freq': 2, 'min_child_samples': 78}. Best is trial 60 with value: 1.5576307654501302.
[I 2020-11-01 12:28:29,017] Trial 62 finished with value: 1.5781215145568417 and parameters: {'learning_rate': 0.01383130268931879, 'lambda_l1': 0.006435576269095933, 'lambda_l2': 0.11424141482607778, 'num_leave

[I 2020-11-01 12:28:56,561] Trial 81 finished with value: 1.5614434188398014 and parameters: {'learning_rate': 0.025496461178673598, 'lambda_l1': 0.00019693365299983123, 'lambda_l2': 0.5855214556412324, 'num_leaves': 104, 'feature_fraction': 0.9665847907972551, 'bagging_fraction': 0.5610797957819306, 'bagging_freq': 1, 'min_child_samples': 98}. Best is trial 71 with value: 1.5563348228165959.
[I 2020-11-01 12:28:57,585] Trial 82 finished with value: 1.5630822293330087 and parameters: {'learning_rate': 0.036912012298094855, 'lambda_l1': 0.000755492326902952, 'lambda_l2': 0.21876145811915276, 'num_leaves': 157, 'feature_fraction': 0.41895871961375153, 'bagging_fraction': 0.46958475160108937, 'bagging_freq': 1, 'min_child_samples': 91}. Best is trial 71 with value: 1.5563348228165959.
[I 2020-11-01 12:28:58,772] Trial 83 finished with value: 1.5556265166207819 and parameters: {'learning_rate': 0.02735521011325212, 'lambda_l1': 4.607847562654462e-05, 'lambda_l2': 2.205644940115449e-07, 'nu

...

[I 2020-11-01 12:32:22,341] Trial 224 finished with value: 1.559734758995338 and parameters: {'learning_rate': 0.023650582296658278, 'lambda_l1': 0.0009487294507885721, 'lambda_l2': 2.564232198839847e-05, 'num_leaves': 153, 'feature_fraction': 0.7214214538962884, 'bagging_fraction': 0.5564692131145086, 'bagging_freq': 5, 'min_child_samples': 76}. Best is trial 183 with value: 1.5432650753984853.
[I 2020-11-01 12:32:23,699] Trial 225 finished with value: 1.5572398122580664 and parameters: {'learning_rate': 0.02490991925295872, 'lambda_l1': 0.001549456444380163, 'lambda_l2': 3.8796512572598826e-05, 'num_leaves': 149, 'feature_fraction': 0.7343751458973512, 'bagging_fraction': 0.5357091446875144, 'bagging_freq': 5, 'min_child_samples': 77}. Best is trial 183 with value: 1.5432650753984853.
[I 2020-11-01 12:32:25,184] Trial 226 finished with value: 1.5579324215278496 and parameters: {'learning_rate': 0.02823399786663136, 'lambda_l1': 0.0005879369210308402, 'lambda_l2': 2.0648264101433783e-

[I 2020-11-01 12:32:55,220] Trial 244 finished with value: 1.5504383249235407 and parameters: {'learning_rate': 0.024647675633821576, 'lambda_l1': 0.00047007650000706856, 'lambda_l2': 1.052071663712404e-05, 'num_leaves': 150, 'feature_fraction': 0.716051832369641, 'bagging_fraction': 0.5128689900290374, 'bagging_freq': 5, 'min_child_samples': 80}. Best is trial 183 with value: 1.5432650753984853.
[I 2020-11-01 12:32:56,637] Trial 245 finished with value: 1.5618384100850677 and parameters: {'learning_rate': 0.031551734846266874, 'lambda_l1': 0.0004243401069020761, 'lambda_l2': 8.951960438746686e-06, 'num_leaves': 149, 'feature_fraction': 0.712701124772536, 'bagging_fraction': 0.5162177445798309, 'bagging_freq': 5, 'min_child_samples': 80}. Best is trial 183 with value: 1.5432650753984853.
[I 2020-11-01 12:32:58,056] Trial 246 finished with value: 1.5533372208219278 and parameters: {'learning_rate': 0.028545474441136972, 'lambda_l1': 0.00033831976878703383, 'lambda_l2': 1.193286819653655

[I 2020-11-01 12:33:23,463] Trial 264 finished with value: 1.5523288528764279 and parameters: {'learning_rate': 0.030135164111810664, 'lambda_l1': 0.00018460307222279953, 'lambda_l2': 2.852294194138224e-06, 'num_leaves': 158, 'feature_fraction': 0.6721887208684776, 'bagging_fraction': 0.49674533001867616, 'bagging_freq': 5, 'min_child_samples': 81}. Best is trial 183 with value: 1.5432650753984853.
[I 2020-11-01 12:33:24,801] Trial 265 finished with value: 1.5462225091469304 and parameters: {'learning_rate': 0.024358967392106686, 'lambda_l1': 0.00031741739090921034, 'lambda_l2': 5.718766525773089e-06, 'num_leaves': 144, 'feature_fraction': 0.682125391899365, 'bagging_fraction': 0.5074082113502741, 'bagging_freq': 5, 'min_child_samples': 83}. Best is trial 183 with value: 1.5432650753984853.
[I 2020-11-01 12:33:26,056] Trial 266 finished with value: 1.552188867738162 and parameters: {'learning_rate': 0.0247563445240811, 'lambda_l1': 0.0002590308315360294, 'lambda_l2': 6.265993654139622e

[I 2020-11-01 12:34:00,397] Trial 284 finished with value: 1.5522795079207596 and parameters: {'learning_rate': 0.027114506530601417, 'lambda_l1': 0.00010311303618790699, 'lambda_l2': 3.2030884561390794e-06, 'num_leaves': 139, 'feature_fraction': 0.6878456682677166, 'bagging_fraction': 0.4871557191461531, 'bagging_freq': 5, 'min_child_samples': 81}. Best is trial 183 with value: 1.5432650753984853.
[I 2020-11-01 12:34:02,325] Trial 285 finished with value: 1.6310510450913953 and parameters: {'learning_rate': 0.08644406735934432, 'lambda_l1': 0.00012619939730719673, 'lambda_l2': 0.00020391552657480065, 'num_leaves': 136, 'feature_fraction': 0.6558195496039727, 'bagging_fraction': 0.8120992259913421, 'bagging_freq': 5, 'min_child_samples': 83}. Best is trial 183 with value: 1.5432650753984853.
[I 2020-11-01 12:34:03,667] Trial 286 finished with value: 1.5598614408540947 and parameters: {'learning_rate': 0.018183447575698737, 'lambda_l1': 0.00024901920775966083, 'lambda_l2': 2.24031943634

['../optuna_checkpoints/lgbm_study_with_bands_and_low_q.pkl']

In [64]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 300
Best trial:
  Value: 1.520475825149673
  Params: 
    learning_rate: 0.028263428345419755
    lambda_l1: 0.0003960483729716468
    lambda_l2: 1.439202199727957e-08
    num_leaves: 131
    feature_fraction: 0.5612537746206463
    bagging_fraction: 0.6881923376572622
    bagging_freq: 2
    min_child_samples: 96


In [69]:
rfr_study = optuna.create_study(direction="minimize")
# study = joblib.load('../optuna_checkpoints/study.pkl')
rfr_study.optimize(objective, n_trials=300)
joblib.dump(rfr_study, '../optuna_checkpoints/rfr_study_w_lq.pkl')

[I 2020-11-01 12:45:14,026] A new study created in memory with name: no-name-d24a2ee5-3228-4c5b-84d1-40e210e9c96e
[I 2020-11-01 12:45:19,038] Trial 0 finished with value: 1.5617013401634385 and parameters: {'n_estimators': 125, 'max_features': 17, 'min_samples_split': 10, 'min_samples_leaf': 18, 'max_samples': 0.6089894403814872}. Best is trial 0 with value: 1.5617013401634385.
[I 2020-11-01 12:45:28,453] Trial 1 finished with value: 1.5908647627433354 and parameters: {'n_estimators': 175, 'max_features': 14, 'min_samples_split': 5, 'min_samples_leaf': 3, 'max_samples': 0.8713608407997044}. Best is trial 0 with value: 1.5617013401634385.
[I 2020-11-01 12:45:43,004] Trial 2 finished with value: 1.5630537385623153 and parameters: {'n_estimators': 250, 'max_features': 24, 'min_samples_split': 11, 'min_samples_leaf': 16, 'max_samples': 0.8556692602236085}. Best is trial 0 with value: 1.5617013401634385.
[I 2020-11-01 12:45:49,719] Trial 3 finished with value: 1.5859418023674987 and paramet

[I 2020-11-01 12:48:53,784] Trial 29 finished with value: 1.5639920658132735 and parameters: {'n_estimators': 125, 'max_features': 16, 'min_samples_split': 10, 'min_samples_leaf': 18, 'max_samples': 0.6352768243129575}. Best is trial 11 with value: 1.5517552888309576.
[I 2020-11-01 12:49:01,546] Trial 30 finished with value: 1.5721912014898236 and parameters: {'n_estimators': 125, 'max_features': 23, 'min_samples_split': 9, 'min_samples_leaf': 9, 'max_samples': 0.8142881786895593}. Best is trial 11 with value: 1.5517552888309576.
[I 2020-11-01 12:49:13,167] Trial 31 finished with value: 1.5544067956046785 and parameters: {'n_estimators': 225, 'max_features': 30, 'min_samples_split': 2, 'min_samples_leaf': 16, 'max_samples': 0.6047094705272411}. Best is trial 11 with value: 1.5517552888309576.
[I 2020-11-01 12:49:25,565] Trial 32 finished with value: 1.5578324304256541 and parameters: {'n_estimators': 225, 'max_features': 30, 'min_samples_split': 3, 'min_samples_leaf': 14, 'max_samples'

[I 2020-11-01 12:53:29,186] Trial 59 finished with value: 1.5597833779494565 and parameters: {'n_estimators': 125, 'max_features': 30, 'min_samples_split': 6, 'min_samples_leaf': 20, 'max_samples': 0.6448007918398831}. Best is trial 11 with value: 1.5517552888309576.
[I 2020-11-01 12:53:40,556] Trial 60 finished with value: 1.5572297355342537 and parameters: {'n_estimators': 225, 'max_features': 27, 'min_samples_split': 12, 'min_samples_leaf': 14, 'max_samples': 0.633981823549624}. Best is trial 11 with value: 1.5517552888309576.
[I 2020-11-01 12:53:52,115] Trial 61 finished with value: 1.5522115955901776 and parameters: {'n_estimators': 225, 'max_features': 30, 'min_samples_split': 2, 'min_samples_leaf': 16, 'max_samples': 0.6005375220101398}. Best is trial 11 with value: 1.5517552888309576.
[I 2020-11-01 12:54:03,774] Trial 62 finished with value: 1.5579630018741613 and parameters: {'n_estimators': 225, 'max_features': 29, 'min_samples_split': 2, 'min_samples_leaf': 15, 'max_samples'

[I 2020-11-01 12:57:59,631] Trial 89 finished with value: 1.5601264996861879 and parameters: {'n_estimators': 100, 'max_features': 29, 'min_samples_split': 5, 'min_samples_leaf': 19, 'max_samples': 0.6439592248332296}. Best is trial 84 with value: 1.5475413843463057.
[I 2020-11-01 12:58:03,309] Trial 90 finished with value: 1.5940822835418218 and parameters: {'n_estimators': 175, 'max_features': 3, 'min_samples_split': 4, 'min_samples_leaf': 20, 'max_samples': 0.682758518858091}. Best is trial 84 with value: 1.5475413843463057.
[I 2020-11-01 12:58:14,521] Trial 91 finished with value: 1.5535418879595806 and parameters: {'n_estimators': 225, 'max_features': 30, 'min_samples_split': 2, 'min_samples_leaf': 19, 'max_samples': 0.6020536440259241}. Best is trial 84 with value: 1.5475413843463057.
[I 2020-11-01 12:58:25,788] Trial 92 finished with value: 1.5575121018685247 and parameters: {'n_estimators': 225, 'max_features': 28, 'min_samples_split': 3, 'min_samples_leaf': 17, 'max_samples': 

[I 2020-11-01 13:02:42,920] Trial 119 finished with value: 1.5714054893938565 and parameters: {'n_estimators': 225, 'max_features': 8, 'min_samples_split': 3, 'min_samples_leaf': 16, 'max_samples': 0.6003537118148289}. Best is trial 84 with value: 1.5475413843463057.
[I 2020-11-01 13:02:54,539] Trial 120 finished with value: 1.5553354580549388 and parameters: {'n_estimators': 225, 'max_features': 30, 'min_samples_split': 4, 'min_samples_leaf': 19, 'max_samples': 0.633526703514051}. Best is trial 84 with value: 1.5475413843463057.
[I 2020-11-01 13:03:03,855] Trial 121 finished with value: 1.5537130369309955 and parameters: {'n_estimators': 175, 'max_features': 30, 'min_samples_split': 3, 'min_samples_leaf': 18, 'max_samples': 0.6237989478220077}. Best is trial 84 with value: 1.5475413843463057.
[I 2020-11-01 13:03:10,905] Trial 122 finished with value: 1.5625400319368787 and parameters: {'n_estimators': 125, 'max_features': 29, 'min_samples_split': 3, 'min_samples_leaf': 17, 'max_sample

...

[I 2020-11-01 13:18:22,558] Trial 209 finished with value: 1.5519790549852195 and parameters: {'n_estimators': 225, 'max_features': 29, 'min_samples_split': 4, 'min_samples_leaf': 18, 'max_samples': 0.6144608370969804}. Best is trial 199 with value: 1.547446128005759.
[I 2020-11-01 13:18:33,764] Trial 210 finished with value: 1.555239122892224 and parameters: {'n_estimators': 225, 'max_features': 30, 'min_samples_split': 3, 'min_samples_leaf': 19, 'max_samples': 0.6004899847003495}. Best is trial 199 with value: 1.547446128005759.
[I 2020-11-01 13:18:44,619] Trial 211 finished with value: 1.5570254019434617 and parameters: {'n_estimators': 225, 'max_features': 28, 'min_samples_split': 4, 'min_samples_leaf': 19, 'max_samples': 0.6242150699640657}. Best is trial 199 with value: 1.547446128005759.
[I 2020-11-01 13:18:55,774] Trial 212 finished with value: 1.560258878518424 and parameters: {'n_estimators': 225, 'max_features': 29, 'min_samples_split': 4, 'min_samples_leaf': 19, 'max_sample

[I 2020-11-01 13:23:24,968] Trial 239 finished with value: 1.5544442553425473 and parameters: {'n_estimators': 100, 'max_features': 29, 'min_samples_split': 3, 'min_samples_leaf': 20, 'max_samples': 0.6165179642797882}. Best is trial 199 with value: 1.547446128005759.
[I 2020-11-01 13:23:36,003] Trial 240 finished with value: 1.5511068263539023 and parameters: {'n_estimators': 225, 'max_features': 29, 'min_samples_split': 4, 'min_samples_leaf': 18, 'max_samples': 0.6001405952375598}. Best is trial 199 with value: 1.547446128005759.
[I 2020-11-01 13:23:47,085] Trial 241 finished with value: 1.5545136334577365 and parameters: {'n_estimators': 225, 'max_features': 29, 'min_samples_split': 4, 'min_samples_leaf': 18, 'max_samples': 0.6062377820798011}. Best is trial 199 with value: 1.547446128005759.
[I 2020-11-01 13:24:01,735] Trial 242 finished with value: 1.5780566003812537 and parameters: {'n_estimators': 225, 'max_features': 28, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_sampl

[I 2020-11-01 13:28:50,716] Trial 269 finished with value: 1.5529742163585478 and parameters: {'n_estimators': 200, 'max_features': 29, 'min_samples_split': 4, 'min_samples_leaf': 20, 'max_samples': 0.6307389607082331}. Best is trial 199 with value: 1.547446128005759.
[I 2020-11-01 13:29:00,596] Trial 270 finished with value: 1.5538371059708953 and parameters: {'n_estimators': 200, 'max_features': 29, 'min_samples_split': 3, 'min_samples_leaf': 20, 'max_samples': 0.6251676719784821}. Best is trial 199 with value: 1.547446128005759.
[I 2020-11-01 13:29:10,945] Trial 271 finished with value: 1.553141072044495 and parameters: {'n_estimators': 200, 'max_features': 30, 'min_samples_split': 4, 'min_samples_leaf': 20, 'max_samples': 0.6188139612874183}. Best is trial 199 with value: 1.547446128005759.
[I 2020-11-01 13:29:22,443] Trial 272 finished with value: 1.5523461742501525 and parameters: {'n_estimators': 200, 'max_features': 28, 'min_samples_split': 3, 'min_samples_leaf': 20, 'max_sampl

[I 2020-11-01 13:34:29,806] Trial 299 finished with value: 1.5533171344175445 and parameters: {'n_estimators': 225, 'max_features': 28, 'min_samples_split': 3, 'min_samples_leaf': 19, 'max_samples': 0.6003692474239175}. Best is trial 199 with value: 1.547446128005759.


['../optuna_checkpoints/rfr_study_w_lq.pkl']

In [70]:
# RFR
print("Number of finished trials: {}".format(len(rfr_study.trials)))
print("Best trial:")
trial = rfr_study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 300
Best trial:
  Value: 1.547446128005759
  Params: 
    n_estimators: 225
    max_features: 29
    min_samples_split: 4
    min_samples_leaf: 19
    max_samples: 0.6005951075683523


In [44]:
best_lgbm = study.best_params

In [ ]:
best_rfr = rfr_study.best_params

In [11]:
from optuna.visualization import plot_contour, plot_optimization_history, plot_parallel_coordinate, plot_slice

In [13]:
plot_optimization_history(study)

In [14]:
plot_parallel_coordinate(study)

In [15]:
plot_slice(study)

# LR

In [57]:
def validate_cv(model):
    kf = KFold(n_splits=7)
    xval_err = 0
    for train, test in kf.split(X):
        model.fit(X[train],y.iloc[train])
        pred = model.predict(X[test])
        err = mean_squared_error(y.iloc[test], pred)
        xval_err+=err
    rmse_cv = xval_err/10
    return rmse_cv

In [49]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [53]:
X = scaler.fit_transform(X = X)

In [58]:
model = LR.LinearRegression()
validate_cv(model)

1.8035710237749083